# Introduction to Regression with Scikit-Learn and Statsmodels
In this notebook, we will explore two popular Python libraries for performing linear regression: scikit-learn and statsmodels. We'll perform the same task using both libraries, compare the approaches, and calculate key metrics like R² and MSE to evaluate our models.

Download the pydataset library

In [13]:
%%capture
%pip install pydataset

In [14]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.formula.api as smf ## This is a new library we will be learning about
from pydataset import data

Let's reuse the Professor salary data.

In [15]:
prof = pd.DataFrame(data("Salaries"))
prof.head(10)

,rank,discipline,yrs.since.phd,yrs.service,sex,salary
1,Prof,B,19,18,Male,139750
2,Prof,B,20,16,Male,173200
3,AsstProf,B,4,3,Male,79750
4,Prof,B,45,39,Male,115000
5,Prof,B,40,41,Male,141500
6,AssocProf,B,6,6,Male,97000
7,Prof,B,30,23,Male,175000
8,Prof,B,45,45,Male,147765
9,Prof,B,21,20,Male,119250
10,Prof,B,18,18,Female,129000


### Split the data into train and test sets
We'll use the `train_test_split` function from scikit-learn for this.

In [16]:
X = pd.get_dummies(prof[["yrs.service","yrs.since.phd","discipline","sex","rank"]],drop_first = True)
y = prof.salary

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Linear Regression with Scikit-Learn

In [17]:
# Initialize the LinearRegression model
mod_sklearn = LinearRegression()

# Fit the model to the training data
mod_sklearn.fit(X_train, y_train)

# Make predictions on the test data
y_pred_sklearn = mod_sklearn.predict(X_test)

# Print the coefficients
print('intercept:', mod_sklearn.intercept_)
dict(zip(X.columns,mod_sklearn.coef_))


intercept: 76061.88884465527


{'yrs.service': -307.88522912168565,
 'yrs.since.phd': 506.78274727003304,
 'discipline_B': 16055.497437933664,
 'sex_Male': 4365.982288697043,
 'rank_AsstProf': -11506.830579395231,
 'rank_Prof': 32699.54539854732}

In [18]:
# Calculate R² and rMSE for scikit-learn
r2_sklearn = r2_score(y_test, y_pred_sklearn)
rmse_sklearn = np.sqrt(mean_squared_error(y_test, y_pred_sklearn))

# Print scikit-learn results
print(f'Scikit-learn Linear Regression R²: {r2_sklearn:.4f}')
print(f'Scikit-learn Linear Regression rMSE: {rmse_sklearn:.4f}')

Scikit-learn Linear Regression R²: 0.2387
Scikit-learn Linear Regression rMSE: 24181.9207


### Linear Regression with Statsmodels
Now, let's perform the same regression using `statsmodels`.

In [19]:
# Difference #0 : Variable names need to match Python variable name conventions
# Print column names before the change
print("Before change:")
print(prof.columns)

# Rename columns to replace '.' with '_'
prof_clean = prof.rename(columns=lambda x: x.replace('.', '_'))

# Print column names after the change
print("After change:")
print(prof_clean.columns)


Before change:
Index(['rank', 'discipline', 'yrs.since.phd', 'yrs.service', 'sex', 'salary'], dtype='object')
After change:
Index(['rank', 'discipline', 'yrs_since_phd', 'yrs_service', 'sex', 'salary'], dtype='object')


In [24]:
# DIFFERENCE #1 : There is a way to write formulas in statmodels where you don't need to
# create seperate objects for X and y so we will split the full data frame
prof_train, prof_test = train_test_split(prof_clean, test_size=0.2, random_state=42)

# DIFFERENCE #2: You don't need to specifically create the dummies of your variables


# DIFFERENCE #3: We use a formula notation which has the target variable on the left side,
# followed by ~ followed by predictors that are separated by a +
formula = 'salary ~ yrs_service + yrs_since_phd + discipline + sex + rank'

# The syntax here uses the formula, and then the data set where the formula
# is getting its variable names from. Here we go ahead and fit it in one step.
model_sm = smf.ols(formula, data=prof_train).fit()

In [21]:
# NOT DIFFERENCE #1: The coefficients will be the same (they may be slightly different when you
# split the data randomly both times, but if we used the same splits, they would be the same)
print("Coefficients:")
print(model_sm.params)

Coefficients:
Intercept           76061.888845
discipline[T.B]     16055.497438
sex[T.Male]          4365.982289
rank[T.AsstProf]   -11506.830579
rank[T.Prof]        32699.545399
yrs_service          -307.885229
yrs_since_phd         506.782747
dtype: float64


In [22]:
#NOT DIFFERENCE #2: R^2 and rMSE should be the same

# Make predictions on the test data
y_pred_sm = model_sm.predict(prof_test)
y_test = prof_test.salary

# Calculate R² and MSE for statsmodels
r2_sm = r2_score(y_test, y_pred_sm)
rmse_sm = np.sqrt(mean_squared_error(y_test, y_pred_sm))

# Print statsmodels results
print(f'Statsmodels Linear Regression R²: {r2_sm:.4f}')
print(f'Statsmodels Linear Regression MSE: {rmse_sm:.4f}')

Statsmodels Linear Regression R²: 0.2387
Statsmodels Linear Regression MSE: 24181.9207


The biggest difference between the two as far as the output goes, is that statsmodels provides us with a lot more information we can use for things like uncertainty and errors. We will look at a lot of these numbers to understand them in the next few weeks.

In [23]:
# BIG DIFFERENCE #1
model_sm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 salary   R-squared:                       0.492
Model:                            OLS   Adj. R-squared:                  0.482
Method:                 Least Squares   F-statistic:                     50.04
Date:                Mon, 21 Oct 2024   Prob (F-statistic):           7.69e-43
Time:                        12:51:06   Log-Likelihood:                -3619.2
No. Observations:                 317   AIC:                             7252.
Df Residuals:                     310   BIC:                             7279.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept         7.606e+04   5695.823     13.354      0.000    6.49e+04    8.73e+04
discipline[T.B]   1.606e+04   2574.948      6.235      0.000     1.1e+04    2.11e+04
sex[T.Male]       4365.9823   4548.013      0.960      0.338   -4582.897    1.33e+04
rank[T.AsstProf] -1.151e+04   4586.433     -2.509      0.013   -2.05e+04   -2482.355
rank[T.Prof]       3.27e+04   3875.540      8.437      0.000    2.51e+04    4.03e+04
yrs_service       -307.8852    233.033     -1.321      0.187    -766.411     150.641
yrs_since_phd      506.7827    265.463      1.909      0.057     -15.554    1029.119
==============================================================================
Omnibus:                       30.781   Durbin-Watson:                   1.835
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               45.038
Skew:                           0.647   Prob(JB):                     1.66e-10
Kurtosis:                       4.317   Cond. No.                         190.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Comparison
Both scikit-learn and statsmodels allow us to perform linear regression, but they approach the problem differently.
In scikit-learn, the focus is more on machine learning techniques, and the interface is designed for general-purpose model fitting, not just regression.
Statsmodels, on the other hand, is specifically designed for statistical analysis and provides more in-depth information about the model (such as p-values, confidence intervals, etc.).
Both libraries have their own strengths, and the choice depends on the task at hand.